In [1]:
import tensorflow as tf
import numpy as np
import time
import psutil
import os
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import pandas as pd

In [2]:
# Disable TensorFlow Caching
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=0"

# Ensure limited GPU memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Batch sizes for testing
BATCH_SIZES = [1, 4, 8, 16, 32, 64]
NUM_RUNS = 5  # Runs per batch size

# Load NASNetMobile model
base_model = NASNetMobile(weights='imagenet', input_shape=(224, 224, 3))
layer_outputs = [layer.output for layer in base_model.layers]
model = Model(inputs=base_model.input, outputs=layer_outputs)

# Precompile TensorFlow function to avoid extra optimizations
@tf.function(experimental_relax_shapes=True)
def run_inference(input_tensor):
    return model(input_tensor)

2025-02-05 12:35:03.714450: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-05 12:35:03.714465: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-05 12:35:03.714469: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-02-05 12:35:03.714481: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-05 12:35:03.714490: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Load CIFAR-100 dataset
def load_cifar100(batch_size):
    dataset = tfds.load("cifar100", split="train", as_supervised=True)
    dataset = dataset.map(lambda x, y: (tf.image.resize(x, [224, 224]), y))  # Resize to 224x224
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [4]:
# Function to measure inference time and memory
def benchmark_model(device, batch_size, dataset_iter):
    print(f"\nRunning on {device} with batch size {batch_size}")
    results = []

    for run in range(NUM_RUNS):
        # Load Dataset
        try:
            input_tensor, _ = next(dataset_iter)  # Get real ImageNet images
        except StopIteration:
            dataset_iter = iter(dataset)  # Restart dataset if exhausted
            input_tensor, _ = next(dataset_iter)
        
        with tf.device(device):
            tensor_sizes = {}

            # Warm-up to avoid cold start latency
            _ = run_inference(input_tensor)

            start_time = time.perf_counter()

            # Measure Layer wise tensor size
            output = run_inference(input_tensor)
            
            # Measure per-layer execution time
            for idx, layer in enumerate(model.layers):
                # Calculate tensor size in MB (float32 = 4 bytes)
                tensor_size = np.prod(output[idx].shape) * 4 / (1024 ** 2)
                tensor_sizes[layer.name] = tensor_size
            
            total_time = time.perf_counter() - start_time
    
            results.append({
                "device": device,
                "batch_size": batch_size,
                "total_time": total_time
            })

            # Manually clear TensorFlow cache after each batch run
            del input_tensor
            tf.keras.backend.clear_session

    return results

## Experiment Logging

In [5]:
# Run benchmarks for CPU and GPU (if available)
devices = ["/CPU:0"]
if gpus:
    devices.append("/GPU:0")

final_results = []

for device in devices:    
    for batch_size in BATCH_SIZES:
        dataset = load_cifar100(batch_size)
        dataset_iter = iter(dataset)
        try:
            tf.profiler.experimental.start(f"logs/Noise/{device[1:4]}/{batch_size}")
            final_results.extend(benchmark_model(device, batch_size, dataset_iter))
        finally:
            tf.profiler.experimental.stop()

2025-02-05 12:35:06.282156: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 12:35:06.282166: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /CPU:0 with batch size 1


2025-02-05 12:35:07.925477: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-02-05 12:35:09.015231: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 12:35:09.090038: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 12:35:09.090634: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/CPU/1/plugins/profile/2025_02_05_12_35_09/C17586.xplane.pb
2025-02-05 12:35:09.142114: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().


Running on /CPU:0 with batch size 4


2025-02-05 12:35:12.333978: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 12:35:12.415352: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 12:35:12.415517: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/CPU/4/plugins/profile/2025_02_05_12_35_12/C17586.xplane.pb
2025-02-05 12:35:12.469688: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 12:35:12.473915: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 12:

...

2025-02-05 12:36:00.029022: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 12:36:00.063728: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 12:36:00.064027: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/GPU/16/plugins/profile/2025_02_05_12_36_00/C17586.xplane.pb
2025-02-05 12:36:00.111602: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 12:36:00.112452: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 12


Running on /GPU:0 with batch size 32


2025-02-05 12:36:28.113694: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 12:36:28.180049: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 12:36:28.185056: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/GPU/32/plugins/profile/2025_02_05_12_36_28/C17586.xplane.pb
2025-02-05 12:36:28.258782: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 12:36:28.261156: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 12


Running on /GPU:0 with batch size 64


2025-02-05 12:36:46.460702: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 12:36:46.633820: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 12:36:46.644093: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Noise/GPU/64/plugins/profile/2025_02_05_12_36_46/C17586.xplane.pb


## Results

In [6]:
df = pd.DataFrame(final_results)
df.head()

,device,batch_size,total_time
0,/CPU:0,1,0.049458
1,/CPU:0,1,0.041017
2,/CPU:0,1,0.044016
3,/CPU:0,1,0.043136
4,/CPU:0,1,0.039686


In [7]:
df.groupby(['device', 'batch_size']).agg({'total_time': ['min', 'mean', 'var', 'max']})

total_time                              
                         min      mean       var       max
device batch_size                                         
/CPU:0 1            0.039686  0.043463  0.000014  0.049458
       4            0.072530  0.080470  0.000039  0.086722
       8            0.105362  0.120782  0.000095  0.132077
       16           0.168056  0.233052  0.004244  0.338702
       32           0.301132  0.317109  0.000257  0.343161
       64           0.588998  1.644711  0.382357  2.208807
/GPU:0 1            0.089570  0.113725  0.002039  0.193980
       4            0.085739  0.165564  0.011512  0.289432
       8            0.086781  0.118002  0.004153  0.233234
       16           0.093115  0.336902  0.135496  0.938201
       32           0.154851  0.465091  0.072001  0.754363
       64           1.085353  1.771882  0.530368  2.934874